In [1]:
import streamlit as st
import pandas as pd
import requests
import os

# URL base de la API de Socrata (SECOP)
BASE_URL = "https://www.datos.gov.co/resource/rpmr-utcd.json"

# Ruta donde se guardarán los datos localmente
DATA_PATH = "data/secop.csv"

def download_data(limit: int = 50000) -> pd.DataFrame:
    """
    Descarga datos desde la API de SECOP y devuelve un DataFrame.
    También puede guardar localmente el archivo para futuras consultas.
    """
    st.info(f"⏳ Descargando datos... (máximo {limit} registros)")
    try:
        response = requests.get(f"{BASE_URL}?$limit={limit}")
        response.raise_for_status()
        data = response.json()
        df = pd.DataFrame(data)
        
        # Guardar localmente para no descargar cada vez
        os.makedirs("data", exist_ok=True)
        df.to_csv(DATA_PATH, index=False)
        
        st.success(f"✅ Datos descargados y guardados en '{DATA_PATH}'")
        return df
    except requests.exceptions.RequestException as e:
        st.error(f"Error al descargar datos: {e}")
        return pd.DataFrame()

def load_local_data() -> pd.DataFrame:
    """
    Carga datos desde el CSV local si existe, para ahorrar tiempo.
    """
    if os.path.exists(DATA_PATH):
        st.info("📂 Cargando datos desde el archivo local...")
        return pd.read_csv(DATA_PATH)
    else:
        st.warning("⚠ No se encontró un archivo local, descargando datos...")
        return download_data()

def show_data_tab():
    """
    Pestaña de Streamlit para cargar y mostrar datos.
    """
    st.title("📥 Carga de Datos - SECOP II")
    st.markdown(
        "En esta sección puedes cargar los datos del SECOP II, "
        "ya sea desde un archivo local o descargándolos directamente de la API."
    )
    
    opcion = st.radio("¿Cómo deseas cargar los datos?", ["Archivo local", "Descargar de API"])
    
    if opcion == "Archivo local":
        df = load_local_data()
    else:
        limit = st.slider("Cantidad de registros a descargar", 1000, 100000, 50000, step=1000)
        if st.button("Descargar datos"):
            df = download_data(limit)
        else:
            df = pd.DataFrame()
    
    if not df.empty:
        st.subheader("Vista previa de los datos")
        st.dataframe(df.head(20))
        st.write(f"**Total de registros cargados:** {len(df):,}")
    else:
        st.warning("No hay datos para mostrar.")


ModuleNotFoundError: No module named 'streamlit'